In [1]:
import pandas as pd
import matplotlib.pyplot as plt

df_hits = pd.read_csv("additional/hits_dataset.csv", delimiter="\t")
df_hits["hit"] = 1
df_hits

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit
0,3e9HZxeyfWwjeyPAMmWSSQ,"thank u, next",['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],87,True,Solo,11,1,79,...,0.22900,0.717,0.653,0.000000,0.1010,-5.634,0.0658,0.412,106.966,1
1,5p7ujcrUXASCNwRaWNHR1C,Without Me,['26VFTg2z8YR0cCuwLzESi2'],['Halsey'],87,True,Solo,1,1,79,...,0.29700,0.752,0.488,0.000009,0.0936,-7.050,0.0705,0.533,136.041,1
2,2xLMifQCjDGFmkHkpNLD9h,SICKO MODE,['0Y5tJX1MQlPlqiwlOH1tJY'],['Travis Scott'],86,True,Solo,3,1,79,...,0.00513,0.834,0.730,0.000000,0.1240,-3.714,0.2220,0.446,155.008,1
3,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,"['246dkjvS1zLTtiykXe5h60', '1zNqQNIdeOUZHb8zbZ...","['Post Malone', 'Swae Lee']",92,False,Collaboration,2,2,79,...,0.55600,0.760,0.479,0.000000,0.0703,-5.574,0.0466,0.913,89.911,1
4,1rqqCSm0Qe4I9rUvWncaom,High Hopes,['20JZFwl6HVl6yg8a4H3ZqK'],['Panic! At The Disco'],86,False,Solo,4,1,78,...,0.19300,0.579,0.904,0.000000,0.0640,-2.729,0.0618,0.681,82.014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11954,5mz9pQZZXNpAw9CdQ7Bk8q,Don't Pity Me - Mono; 2003 Remaster,['6lHC2EQMEMZiEmSfFloarn'],['Peter And Gordon'],8,False,Solo,1,1,78,...,0.61900,0.500,0.654,0.000000,0.3320,-8.633,0.0394,0.403,107.326,1
11955,2H9CKpZiLDF223BbwehpDF,Hush Hush; Hush Hush - Main,['6wPhSqRtPu1UhRCDX5yaDJ'],['The Pussycat Dolls'],57,False,Solo,1,1,59,...,0.10500,0.609,0.790,0.000000,0.3140,-3.787,0.0663,0.664,129.334,1
11956,7zqM7Wm2xJLZ50qAnTLoHD,"Lions, Tigers & Bears",['7gSjFKpVmDgC2MMsnN8CYq'],['Jazmine Sullivan'],49,False,Solo,4,1,71,...,0.07880,0.576,0.515,0.000000,0.1180,-6.178,0.0295,0.407,95.406,1
11957,4DPlKVZ6R1EHLfnIsy9ouh,The Weight,"['0rXI0q8Cahq6numvPlloaq', '3RwQ26hR2tJtA8F9p2...","['Diana Ross & The Supremes', 'The Temptations']",2,False,Collaboration,23,2,79,...,0.37000,0.566,0.607,0.000000,0.6710,-7.532,0.0357,0.765,91.378,1


In [2]:
df_nonhits = pd.read_csv("additional/nonhits_dataset.csv", delimiter="\t")
df_nonhits["hit"] = 0
df_nonhits

C:\Users\lasse\AppData\Local\Temp\ipykernel_24340\2533836516.py:1: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nonhits = pd.read_csv("additional/nonhits_dataset.csv", delimiter="\t")


,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit
0,1TEL6MlSSVLSdhOSddidlJ,needy,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],76,True,Solo,2,1,79,...,0.78,0.647,0.309,7.41e-06,0.202,-7.948,0.0366,0.195,87.045,0
1,4uTvPEr01pjTbZgl7jcKBD,NASA,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],76,False,Solo,3,1,79,...,0.451,0.747,0.458,0,0.252,-6.891,0.303,0.47,75.029,0
2,2hloaUoRonYssMuqLCBLTX,bloodline,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],74,True,Solo,4,1,79,...,0.0815,0.758,0.665,0.000157,0.216,-5.188,0.0774,0.643,151.976,0
3,3wFLWP0FcIqHK1wb1CPthQ,fake smile,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],73,True,Solo,5,1,79,...,0.329,0.45,0.619,0,0.319,-7.412,0.372,0.427,83.3,0
4,5Il6Oe7lr5XM7A0cWbVQtr,bad idea,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],74,False,Solo,6,1,79,...,0.0268,0.847,0.579,0.0545,0.102,-5.314,0.0719,0.569,137.998,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899063,2tP0ryZParpdjvxubgXhTN,Love Is Here to Stay,"['3Dzj993UEz8Z5ovxuirzFO', '6RCsx4p5smZHYz2P5H...","['Lindsey Buckingham', 'Christine McVie']",16,False,Collaboration,5,2,1,...,0.366,0.446,0.658,2.49e-05,0.751,-8.362,0.0297,0.554,84.74,0
899064,7Ltni80U0w7bmYjXPNzSEa,Heinlein's Planet,['4urvwbid7NFoVes5PTx9lV'],['Tony Carey'],0,False,Solo,5,1,79,...,0.481,0.305,0.456,0.933,0.111,-20.199,0.0634,0.171,96.996,0
899065,1gQClXN1Dq4rj0wBzddcNA,Beds are Burning - Karaoke Lead Vocal Version,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,2,1,79,...,0.0118,0.852,0.648,0,0.0503,-7.189,0.048,0.559,120.15,0
899066,2jhxQrFNdVPkWL28X7ZpeK,Un homme heureux - Version karaoké instrumentale,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,1,1,79,...,0.973,0.664,0.353,0.787,0.0878,-12.303,0.0693,0.284,116.45,0


In [3]:
combined_df = pd.concat([df_hits, df_nonhits], axis=0)
combined_df.replace("-", float("nan"), inplace=True)
# Drop all rows with NaN values
combined_df

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit
0,3e9HZxeyfWwjeyPAMmWSSQ,"thank u, next",['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],87,True,Solo,11,1,79,...,0.229,0.717,0.653,0.0,0.101,-5.634,0.0658,0.412,106.966,1
1,5p7ujcrUXASCNwRaWNHR1C,Without Me,['26VFTg2z8YR0cCuwLzESi2'],['Halsey'],87,True,Solo,1,1,79,...,0.297,0.752,0.488,0.000009,0.0936,-7.05,0.0705,0.533,136.041,1
2,2xLMifQCjDGFmkHkpNLD9h,SICKO MODE,['0Y5tJX1MQlPlqiwlOH1tJY'],['Travis Scott'],86,True,Solo,3,1,79,...,0.00513,0.834,0.73,0.0,0.124,-3.714,0.222,0.446,155.008,1
3,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,"['246dkjvS1zLTtiykXe5h60', '1zNqQNIdeOUZHb8zbZ...","['Post Malone', 'Swae Lee']",92,False,Collaboration,2,2,79,...,0.556,0.76,0.479,0.0,0.0703,-5.574,0.0466,0.913,89.911,1
4,1rqqCSm0Qe4I9rUvWncaom,High Hopes,['20JZFwl6HVl6yg8a4H3ZqK'],['Panic! At The Disco'],86,False,Solo,4,1,78,...,0.193,0.579,0.904,0.0,0.064,-2.729,0.0618,0.681,82.014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899063,2tP0ryZParpdjvxubgXhTN,Love Is Here to Stay,"['3Dzj993UEz8Z5ovxuirzFO', '6RCsx4p5smZHYz2P5H...","['Lindsey Buckingham', 'Christine McVie']",16,False,Collaboration,5,2,1,...,0.366,0.446,0.658,2.49e-05,0.751,-8.362,0.0297,0.554,84.74,0
899064,7Ltni80U0w7bmYjXPNzSEa,Heinlein's Planet,['4urvwbid7NFoVes5PTx9lV'],['Tony Carey'],0,False,Solo,5,1,79,...,0.481,0.305,0.456,0.933,0.111,-20.199,0.0634,0.171,96.996,0
899065,1gQClXN1Dq4rj0wBzddcNA,Beds are Burning - Karaoke Lead Vocal Version,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,2,1,79,...,0.0118,0.852,0.648,0,0.0503,-7.189,0.048,0.559,120.15,0
899066,2jhxQrFNdVPkWL28X7ZpeK,Un homme heureux - Version karaoké instrumentale,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,1,1,79,...,0.973,0.664,0.353,0.787,0.0878,-12.303,0.0693,0.284,116.45,0


In [4]:
# Check for NaN values in all columns
nan_columns = combined_df.columns[combined_df.isna().any()].tolist()

# Print columns with NaN values
print("Columns with NaN values:")
print(nan_columns)


Columns with NaN values:
['duration_ms', 'key', 'mode', 'time_signature', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence', 'tempo']


In [5]:
from sklearn.impute import SimpleImputer

combined_df.replace("-", float("nan"), inplace=True)   ## evtl hier schon mean replacement???
#combined_df.dropna(inplace=True)   # drop nan value rows
#Fill missing values with mean for each numeric attribute
imputer = SimpleImputer(strategy='mean')
for col in nan_columns:
    combined_df[col] = imputer.fit_transform(combined_df[[col]])

combined_df

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit
0,3e9HZxeyfWwjeyPAMmWSSQ,"thank u, next",['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],87,True,Solo,11,1,79,...,0.22900,0.717,0.653,0.000000,0.1010,-5.634,0.0658,0.412,106.966,1
1,5p7ujcrUXASCNwRaWNHR1C,Without Me,['26VFTg2z8YR0cCuwLzESi2'],['Halsey'],87,True,Solo,1,1,79,...,0.29700,0.752,0.488,0.000009,0.0936,-7.050,0.0705,0.533,136.041,1
2,2xLMifQCjDGFmkHkpNLD9h,SICKO MODE,['0Y5tJX1MQlPlqiwlOH1tJY'],['Travis Scott'],86,True,Solo,3,1,79,...,0.00513,0.834,0.730,0.000000,0.1240,-3.714,0.2220,0.446,155.008,1
3,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,"['246dkjvS1zLTtiykXe5h60', '1zNqQNIdeOUZHb8zbZ...","['Post Malone', 'Swae Lee']",92,False,Collaboration,2,2,79,...,0.55600,0.760,0.479,0.000000,0.0703,-5.574,0.0466,0.913,89.911,1
4,1rqqCSm0Qe4I9rUvWncaom,High Hopes,['20JZFwl6HVl6yg8a4H3ZqK'],['Panic! At The Disco'],86,False,Solo,4,1,78,...,0.19300,0.579,0.904,0.000000,0.0640,-2.729,0.0618,0.681,82.014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899063,2tP0ryZParpdjvxubgXhTN,Love Is Here to Stay,"['3Dzj993UEz8Z5ovxuirzFO', '6RCsx4p5smZHYz2P5H...","['Lindsey Buckingham', 'Christine McVie']",16,False,Collaboration,5,2,1,...,0.36600,0.446,0.658,0.000025,0.7510,-8.362,0.0297,0.554,84.740,0
899064,7Ltni80U0w7bmYjXPNzSEa,Heinlein's Planet,['4urvwbid7NFoVes5PTx9lV'],['Tony Carey'],0,False,Solo,5,1,79,...,0.48100,0.305,0.456,0.933000,0.1110,-20.199,0.0634,0.171,96.996,0
899065,1gQClXN1Dq4rj0wBzddcNA,Beds are Burning - Karaoke Lead Vocal Version,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,2,1,79,...,0.01180,0.852,0.648,0.000000,0.0503,-7.189,0.0480,0.559,120.150,0
899066,2jhxQrFNdVPkWL28X7ZpeK,Un homme heureux - Version karaoké instrumentale,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,1,1,79,...,0.97300,0.664,0.353,0.787000,0.0878,-12.303,0.0693,0.284,116.450,0


In [6]:
# Check for NaN values in all columns
nan_columns = combined_df.columns[combined_df.isna().any()].tolist()

# Print columns with NaN values
print("Columns with NaN values:")
print(nan_columns)


Columns with NaN values:
[]


In [7]:
from sklearn.model_selection import train_test_split
combined_df['date'] = pd.to_datetime(combined_df['release_date'])
combined_df.sort_values(by="date", inplace=True)

train_df, test_df = train_test_split(combined_df, test_size=0.25, shuffle=False)

In [8]:
# Specify the columns
columns = ['artist_id', 'artist_name']

# Initialize an empty DataFrame
df_artists_train = pd.DataFrame(columns={col: [] for col in columns})

count = 0
artist_set = set()

for _, row in train_df.iterrows():
    current_artist_id_list = eval(row["id_artists"])
    current_artist_name_list = eval(row["name_artists"])
    for cur_art_id, cur_art_name in zip(current_artist_id_list, current_artist_name_list):
        if cur_art_id not in artist_set:
            artist_set.add(cur_art_id)
            df_artists_train.loc[len(df_artists_train)] = [cur_art_id, cur_art_name]
            count += 1

print("amount of artists:", count)
df_artists_train

amount of artists: 18551


,artist_id,artist_name
0,6eVCRmRG2HcfL4yx77A9WK,Jack Jones
1,5NDMothbpdpq2xHqSjrrWn,Faith Evans
2,3z6Uf5IZkN4TogRpRpgD84,Gary Wright
3,2UuKUaomk3bZ0nHM3Y3FdW,Bud Shank
4,6S0GHTqz5sxK5f9HtLXn9q,Garbage
...,...,...
18546,3bj9NXSjQfajNgU9woBnrR,Country Pop All-Stars
18547,7ueug5uU2UKBoGQInMsKQV,Deep Noise
18548,4UD3hwaHMCoJ9GKgIoMzTp,Steve Dafoe
18549,2OdSf9rTLg6GYfoOrK2Kjl,Ron Komie


In [9]:
import networkx as nx

all_artist_ids = artist_set.copy()

G = nx.Graph()

for id_art_cur in all_artist_ids:
    artist_name = df_artists_train[(df_artists_train["artist_id"] == id_art_cur)]["artist_name"]
    artist_info = df_artists_train[df_artists_train["artist_id"] == id_art_cur]
    if not artist_info.empty:  # Check if artist info exists
        # Filter out artists who are not successful
        node_attrs = {"name": artist_name.iloc[0]}
        G.add_node(id_art_cur, **node_attrs)
    else:
        print("Artist info not found for ID:", id_art_cur)

for _, row in train_df.iterrows():
    artist_ids = list(eval(row["id_artists"]))
    artist_names = list(eval(row["name_artists"]))
    # Filter out songs that do not have more than one artist in their execution
    if len(artist_ids) > 1:
        for i in range(len(artist_ids)):
            for j in range(i+1, len(artist_ids)):
                artist_id_1 = str(artist_ids[i])
                lable1 = str(artist_names[i])
                artist_id_2 = str(artist_ids[j])
                lable2 = str(artist_names[j])
                # Check if the edge already exists
                if G.has_edge(artist_id_1, artist_id_2):
                    # If the edge already exists, increment the weight by 1
                    G[artist_id_1][artist_id_2]['weight'] += 2
                else:
                    # Add a new edge with weight 1
                    G.add_edge(artist_id_1, artist_id_2, lable1=lable1, lable2=lable2, weight=2, song_id=str(row["song_id"]), song_name=str(row["song_name"]), explicit=bool(row["explicit"]), song_type=str(row["song_type"]), track_number=int(row["track_number"]), num_artists=int(row["num_artists"]), num_available_markets=int(row["num_available_markets"]), duration_ms=int(row["duration_ms"]), popularity=int(row["popularity"]), release_date=str(row["release_date"]), key = int(row["key"]), mode = int(row["mode"]), time_signature = int(row["time_signature"]), acousticness = float(row["acousticness"]), danceability = float(row["danceability"]), energy = float(row["energy"]), instrumentalness = float(row["instrumentalness"]), liveness = float(row["liveness"]),loudness = float(row["loudness"]), speechiness = float(row["speechiness"]), valence = float(row["valence"]), tempo = float(row["tempo"]), hit = int(row["hit"]))

In [10]:
# Assuming G is your NetworkX graph

# Extract nodes and their attributes from the graph
nodes_with_attributes = [(G.nodes[node], node) for node in G.nodes()]

# Create a DataFrame from the nodes and their attributes
df_nodes_train = pd.DataFrame(nodes_with_attributes, columns=['Attributes', "Spotify ID"])

# Normalize the 'Attributes' column to expand it into separate columns
df_attributes = pd.json_normalize(df_nodes_train['Attributes'])

# Combine the original 'Node' column with the expanded attributes
df_nodes_train = pd.concat([df_attributes, df_nodes_train['Spotify ID']], axis=1)

# Display the DataFrame
df_nodes_train

,name,Spotify ID
0,Love,3Q6OOkfssqoMSTtl11J5Uk
1,Sister Janet Mead,2koUqfFTZqZIaTHxtWmT4W
2,Zac Brown,6tKJaNOY6oNVWcegnMMEtt
3,Jennifer Rush,3fsCWTIuWIRvEGS7gerSJX
4,Chris Cox,1PLakF1tFiv9zn73N2UjaN
...,...,...
18546,Janet,0LwySibLlyu9JUdUFN1ZoS
18547,Bat For Lashes,6l77PmL5iuEEcYjGl8K6s7
18548,J-Scrilla,0XZGO7pKX4VylcUhiznzBJ
18549,Tanto Metro & Devonte,4mVpYIVAOE8sEdYuL8HDpd


In [11]:
df_nodes_train.to_csv("network_created/nodes_real_train.csv", sep='\t', index=False)

In [12]:
# Extract edges and their attributes from the graph
edges_with_attributes = [(u, v, G.edges[u, v]) for u, v in G.edges()]

# Create a DataFrame from the edges and their attributes
df_edges_train = pd.DataFrame(edges_with_attributes, columns=['Source', 'Target', 'Attributes'])

# Normalize the 'Attributes' column to expand it into separate columns
df_edge_attributes = pd.json_normalize(df_edges_train['Attributes'])

# Combine the original 'Attributes' column with the expanded attributes
df_edges_train = pd.concat([df_edges_train[['Source', 'Target']], df_edge_attributes], axis=1)
df_edges_train

,Source,Target,lable1,lable2,weight,song_id,song_name,explicit,song_type,track_number,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit
0,3Q6OOkfssqoMSTtl11J5Uk,7sECLFfpeGHQYRH7Lvc9pA,Love,Arthur Lee,60,2zKXP2wlh5RlzC50LCn5pQ,Andmoreagain - Live,False,Collaboration,3,...,0.13100,0.526,0.470,0.000338,0.7420,-9.823,0.0251,0.331,96.499,0
1,6tKJaNOY6oNVWcegnMMEtt,7dOBabd5O4CvKrg4iriHTM,Kid Rock,Zac Brown,2,3snsImCMTvsx4LuCdbP2Jr,Flyin' High (feat. Zac Brown),False,Collaboration,8,...,0.14600,0.517,0.771,0.000002,0.1110,-3.899,0.0384,0.850,148.780,0
2,6tKJaNOY6oNVWcegnMMEtt,4mxWe1mtYIYfP040G38yvS,Alan Jackson,Zac Brown,2,0r2QTUc1Ky2VIv2PPOCOC5,Dixie Highway,False,Collaboration,7,...,0.06880,0.628,0.868,0.000093,0.1460,-8.176,0.0495,0.506,109.957,0
3,3fsCWTIuWIRvEGS7gerSJX,3PhoLpVuITZKcymswpck5b,Jennifer Rush,Elton John,4,394pkxGNi9boN5zDxJsCsH,Flames of Paradise,False,Collaboration,3,...,0.09640,0.689,0.918,0.000097,0.3040,-6.466,0.0831,0.848,149.862,0
4,3fsCWTIuWIRvEGS7gerSJX,4pU3BpenOZFEBzORx2YBJW,Plácido Domingo,Jennifer Rush,2,7l8JpQ3dGbBfujtXYD4ilC,Till I Loved You,False,Collaboration,17,...,0.30600,0.555,0.618,0.028500,0.2930,-5.861,0.0287,0.368,105.776,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17945,7zQmORVNaWz1cW1KJB2cTX,3RGLhK1IP9jnYFH4BRFJBS,The Clash,Mick Jones,2,216tRx9zfuKKILYJt3t1fi,Rock the Casbah,False,Collaboration,13,...,0.35500,0.775,0.925,0.000000,0.0841,-4.426,0.0547,0.919,129.530,1
17946,2jGkJ4PlZ8t0k5Gamf96kS,0zOuQuYGGcrtDYYy3YFDMr,F.L.Y. (Fast Life Yungstaz),Dear Jayne,4,3VyaQaJx7wTN6GLAP6cLnl,Better Days,True,Collaboration,11,...,0.21300,0.591,0.715,0.000000,0.4090,-5.091,0.0543,0.461,75.005,0
17947,558Kwvm4cFCPWUkrgOoKRI,4tpUmLEVLCGFr93o8hFFIB,Blondie,Smoove-T,2,3ojzxhqcj79ODXmOz7qatA,Call Me - E-Smoove's Beat Vocal Mix,False,Collaboration,8,...,0.01050,0.676,0.945,0.670000,0.1750,-10.465,0.0485,0.782,127.050,0
17948,3rs3EOlJ8jyPpdGiQ9Mhub,6sOhwBjVzpiEBAYEWYwDoX,Monchy & Alexandra,Alexandra,24,76Z8ZjBEb7ottRpj0Bqn3V,Volvere,False,Collaboration,12,...,0.01740,0.931,0.600,0.000027,0.1120,-6.788,0.0634,0.889,127.870,0


In [13]:
# Save edge DataFrame to CSV
df_edges_train.to_csv('network_created/edges_real_train.csv', sep='\t', index=False)

In [14]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)


Number of nodes: 18551
Number of edges: 17950


In [15]:
# Specify the columns
columns = ['artist_id', 'artist_name']

# Initialize an empty DataFrame
df_artists_test = pd.DataFrame(columns={col: [] for col in columns})

count = 0
artist_set = set()

for _, row in test_df.iterrows():
    current_artist_id_list = eval(row["id_artists"])
    current_artist_name_list = eval(row["name_artists"])
    for cur_art_id, cur_art_name in zip(current_artist_id_list, current_artist_name_list):
        if cur_art_id not in artist_set:
            artist_set.add(cur_art_id)
            df_artists_test.loc[len(df_artists_test)] = [cur_art_id, cur_art_name]
            count += 1

print("amount of artists:", count)
df_artists_test

amount of artists: 13209


,artist_id,artist_name
0,5gLIbkXGeoJh0bzeEmvG2n,Underwater
1,1N477jyVWGcvVoRnvOePue,Gustavo Coutinho
2,3BL7Z4GDZQOEnlETaPYV93,Orgel Sound J-pop
3,3OpJxAHfT0KsJzQqdX5SLb,Ginger Tunes
4,59n7uqUFv9tprlcVU9CoXL,Jive Ass Sleepers
...,...,...
13204,69Mj3u4FTUrpyeGNSIaU6F,The J. Geils Band
13205,6piiChdjB4mmBVvOs7rWco,The Hollywood Stars
13206,3lJ6jvOiNY46L2juqXufpf,Changing Faces
13207,6JjTnHt1P7gQXlcFDoI4IC,Jellybean


In [16]:
import networkx as nx

all_artist_ids = artist_set.copy()

G2 = nx.Graph()

for id_art_cur in all_artist_ids:
    artist_name = df_artists_test[(df_artists_test["artist_id"] == id_art_cur)]["artist_name"]
    artist_info = df_artists_test[df_artists_test["artist_id"] == id_art_cur]
    if not artist_info.empty:  # Check if artist info exists
        # Filter out artists who are not successful
        node_attrs = {"name": artist_name.iloc[0]}
        G2.add_node(id_art_cur, **node_attrs)
    else:
        print("Artist info not found for ID:", id_art_cur)

for _, row in test_df.iterrows():
    artist_ids = list(eval(row["id_artists"]))
    artist_names = list(eval(row["name_artists"]))
    # Filter out songs that do not have more than one artist in their execution
    if len(artist_ids) > 1:
        for i in range(len(artist_ids)):
            for j in range(i+1, len(artist_ids)):
                artist_id_1 = str(artist_ids[i])
                lable1 = str(artist_names[i])
                artist_id_2 = str(artist_ids[j])
                lable2 = str(artist_names[j])
                # Check if the edge already exists
                if G2.has_edge(artist_id_1, artist_id_2):
                    # If the edge already exists, increment the weight by 1
                    G2[artist_id_1][artist_id_2]['weight'] += 2
                else:
                    # Add a new edge with weight 1
                    G2.add_edge(artist_id_1, artist_id_2, lable1=lable1, lable2=lable2, weight=2, song_id=str(row["song_id"]), song_name=str(row["song_name"]), explicit=bool(row["explicit"]), song_type=str(row["song_type"]), track_number=int(row["track_number"]), num_artists=int(row["num_artists"]), num_available_markets=int(row["num_available_markets"]), duration_ms=int(row["duration_ms"]), popularity=int(row["popularity"]), release_date=str(row["release_date"]), key = int(row["key"]), mode = int(row["mode"]), time_signature = int(row["time_signature"]), acousticness = float(row["acousticness"]), danceability = float(row["danceability"]), energy = float(row["energy"]), instrumentalness = float(row["instrumentalness"]), liveness = float(row["liveness"]),loudness = float(row["loudness"]), speechiness = float(row["speechiness"]), valence = float(row["valence"]), tempo = float(row["tempo"]), hit = int(row["hit"]))

In [17]:
# Assuming G is your NetworkX graph

# Extract nodes and their attributes from the graph
nodes_with_attributes = [(G2.nodes[node], node) for node in G2.nodes()]

# Create a DataFrame from the nodes and their attributes
df_nodes_test = pd.DataFrame(nodes_with_attributes, columns=['Attributes', "Spotify ID"])

# Normalize the 'Attributes' column to expand it into separate columns
df_attributes = pd.json_normalize(df_nodes_test['Attributes'])

# Combine the original 'Node' column with the expanded attributes
df_nodes_test = pd.concat([df_attributes, df_nodes_test['Spotify ID']], axis=1)

# Display the DataFrame
df_nodes_test

,name,Spotify ID
0,Love,3Q6OOkfssqoMSTtl11J5Uk
1,Skip Marley,4ryoUS0W8qXokfMxrlJt6O
2,Hip Hop Hitmakers,0pRVH1LndRDUbOHUsWf8Lv
3,Lil Flash,069qBEK34YGoX7nSIT74Eg
4,Pablo Rider,3QpOKxHTsQOrSXnlBq10kT
...,...,...
13204,Protophonic,3C92ujpL2KeDXdQBnK4oWr
13205,D. Shawn,6hU4pAtVvo7KpUXwvt4iHC
13206,Justin Crosby,6htnmY9pwmNQ70HC7uAvya
13207,Tanto Metro & Devonte,4mVpYIVAOE8sEdYuL8HDpd


In [18]:
df_nodes_test.to_csv("network_created/nodes_real_test.csv", sep='\t', index=False)

In [19]:
# Extract edges and their attributes from the graph
edges_with_attributes = [(u, v, G2.edges[u, v]) for u, v in G2.edges()]

# Create a DataFrame from the edges and their attributes
df_edges_test = pd.DataFrame(edges_with_attributes, columns=['Source', 'Target', 'Attributes'])

# Normalize the 'Attributes' column to expand it into separate columns
df_edge_attributes = pd.json_normalize(df_edges_test['Attributes'])

# Combine the original 'Attributes' column with the expanded attributes
df_edges_test = pd.concat([df_edges_test[['Source', 'Target']], df_edge_attributes], axis=1)
df_edges_test

,Source,Target,lable1,lable2,weight,song_id,song_name,explicit,song_type,track_number,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit
0,4ryoUS0W8qXokfMxrlJt6O,6jJ0s89eD6GaHleKKya26X,Katy Perry,Skip Marley,2,7qvxFz3JodM0A7xEM7k3YD,Chained To The Rhythm,False,Collaboration,9,...,0.081400,0.446,0.800,0.000000,0.1990,-5.404,0.1740,0.474,189.890,0
1,4ryoUS0W8qXokfMxrlJt6O,5iNrZmtVMtYev5M9yoWpEq,Seeb,Skip Marley,2,3akK9yI6CIMKB2V9eQWRGc,Cruel World,False,Collaboration,1,...,0.163000,0.411,0.861,0.000000,0.2170,-3.627,0.0581,0.413,93.863,0
2,4ryoUS0W8qXokfMxrlJt6O,3QJzdZJYIAcoET1GcfpNGi,Skip Marley,Damian Marley,2,6747tP2tfqp3KgxLVF2524,"That's Not True (feat. Damian ""Jr. Gong"" Marley)",False,Collaboration,1,...,0.150000,0.716,0.727,0.000000,0.1350,-5.399,0.0813,0.809,133.999,0
3,069qBEK34YGoX7nSIT74Eg,13y7CgLHjMVRMDqxdx0Xdo,Gucci Mane,Lil Flash,2,6BDPdwf58rYKPeMscqNnW3,Pari (Feat. Lil Flash),True,Collaboration,11,...,0.262000,0.903,0.400,0.000069,0.1120,-9.738,0.5700,0.203,129.103,0
4,3QpOKxHTsQOrSXnlBq10kT,69MEO1AADKg1IZrq2XLzo5,UB40,Pablo Rider,4,4DXl3ij5d1wxvalqftRZZQ,Bruk Heart Dub,False,Collaboration,2,...,0.007020,0.691,0.476,0.889000,0.1660,-9.165,0.0320,0.560,140.010,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11207,3tCxM1rTj2qUXsiQZxvt2I,6f4y1h4jbvEDZXMojr4hNn,Nappy Roots,Emi Meyer,2,68UBMVGX0c8Fy1b7YXOXdK,Wings,True,Collaboration,9,...,0.457000,0.644,0.626,0.000000,0.3310,-5.194,0.0777,0.375,82.980,0
11208,1gPhS1zisyXr5dHTYZyiMe,7kwCkEJ384PWm0UQW3hxjS,Yella Beezy,Kevin Gates,2,5o4Kal1PbEg8VbVjDoCx80,What I Did (feat. Kevin Gates),True,Collaboration,4,...,0.000397,0.449,0.684,0.000000,0.3660,-7.309,0.2190,0.304,122.935,0
11209,2YrfEkWJvn1wue6JqekigJ,1Bo8Afb2Qbjs4x6kJHyjle,Gigamesh,Daktyl,2,0y3HEldJuR8E2Jztkjr5MB,Everybody's Gotta Live - Daktyl Remix,False,Collaboration,2,...,0.587000,0.401,0.706,0.739000,0.0833,-9.527,0.0958,0.689,73.865,0
11210,1RyvyyTE3xzB2ZywiAwp0i,0rG0AZBscc8S8q1ahIsasI,Future,Juvenile,2,1lxKygrWpySLfb2iKgOKv7,Aintchu,True,Collaboration,3,...,0.027600,0.649,0.684,0.000000,0.6040,-6.715,0.0968,0.501,154.030,0


In [20]:
# Save edge DataFrame to CSV
df_edges_test.to_csv('network_created/edges_real_test.csv', sep='\t', index=False)

In [21]:
num_nodes = G2.number_of_nodes()
num_edges = G2.number_of_edges()
print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)


Number of nodes: 13209
Number of edges: 11212


In [22]:
train_df

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit,date
804676,7kXUEJmfvRXbzxOC0pHQgb,I Can't Believe I'm Losing You,['6eVCRmRG2HcfL4yx77A9WK'],['Jack Jones'],5,False,Solo,12,1,79,...,0.180,0.369,0.000000,0.1500,-9.797,0.0299,0.2240,81.701,0,1995-01-01
465447,1xdRFXv3ytSOdj8GuT3Rge,Thank You Lord (Interlude),['5NDMothbpdpq2xHqSjrrWn'],['Faith Evans'],20,False,Solo,10,1,77,...,0.370,0.284,0.000000,0.5580,-14.443,0.0378,0.6490,130.985,0,1995-01-01
465446,5wlQIBbKIWoB7AnSw09C6C,All This Love,['5NDMothbpdpq2xHqSjrrWn'],['Faith Evans'],26,False,Solo,9,1,77,...,0.605,0.604,0.000153,0.0693,-9.214,0.0480,0.5030,145.483,0,1995-01-01
377956,40uHAEjua1YmyKIJ6k0iVa,Bo Bo Wa,['3z6Uf5IZkN4TogRpRpgD84'],['Gary Wright'],0,False,Solo,8,1,79,...,0.481,0.652,0.008670,0.0397,-10.981,0.0364,0.7950,195.791,0,1995-01-01
465445,2zmMWWgwCXYarkO2SbyPZP,Come Over,['5NDMothbpdpq2xHqSjrrWn'],['Faith Evans'],38,False,Solo,7,1,77,...,0.534,0.563,0.000001,0.5740,-8.975,0.0601,0.4140,124.149,0,1995-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677962,45oru9CcsrV3EWBMl1Wflg,Angels Moon,['7FQ8wLlE6kd6EBtexh0FAD'],['Richy Kicklighter'],0,False,Solo,6,1,79,...,0.580,0.480,0.907000,0.1100,-17.467,0.0317,0.6960,113.312,0,2015-08-19
819520,5AZQNFCdxA3W1nssrtu4Yr,能登半島 Originally Performed By 石川さゆり (オルゴール),['3BL7Z4GDZQOEnlETaPYV93'],['Orgel Sound J-pop'],0,False,Solo,7,1,79,...,0.434,0.498,0.923000,0.1310,-10.614,0.0572,0.0688,119.961,0,2015-08-19
678026,0gvKPCmhJzXa4YgptrNvI4,Up the Hill,['3FRfnhYD8uu3k8Dml7C7Wy'],['Jason Meeks'],0,False,Solo,30,1,79,...,0.813,0.645,0.826000,0.0773,-10.449,0.1390,0.4040,120.051,0,2015-08-19
819519,5FDxZkCXwOOO0CQP3zQUBL,浪花恋しぐれ Originally Performed By 都はるみ・岡千秋 (オルゴール),['3BL7Z4GDZQOEnlETaPYV93'],['Orgel Sound J-pop'],0,False,Solo,6,1,79,...,0.616,0.443,0.909000,0.0893,-9.730,0.0504,0.1430,140.022,0,2015-08-19


In [23]:
test_df

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit,date
677960,0KkzGvp8twn623wysKZ4Th,Return to Ipanema,['5gLIbkXGeoJh0bzeEmvG2n'],['Underwater'],0,False,Solo,4,1,79,...,0.760,0.179,0.783000,0.1040,-21.624,0.0500,0.5980,138.898,0,2015-08-19
677959,6LYpAOMzSqyB2y2mNmqy5E,Foreign Feeling,['1N477jyVWGcvVoRnvOePue'],['Gustavo Coutinho'],0,False,Solo,3,1,79,...,0.430,0.274,0.786000,0.0887,-19.042,0.0307,0.2330,140.009,0,2015-08-19
819517,6EU7nvOyg2O8ld7UAKii7R,どうにもとまらない Originally Performed By 山本リンダ (オルゴール),['3BL7Z4GDZQOEnlETaPYV93'],['Orgel Sound J-pop'],0,False,Solo,4,1,79,...,0.595,0.594,0.919000,0.0993,-8.746,0.0551,0.1740,111.978,0,2015-08-19
677958,32bnbgvJ8FevA9rT2FxGAO,Wave,['3OpJxAHfT0KsJzQqdX5SLb'],['Ginger Tunes'],1,False,Solo,2,1,79,...,0.668,0.414,0.902000,0.1120,-10.019,0.0361,0.0399,121.084,0,2015-08-19
677957,7aMr9p35GaCmdWBmqbpfMa,Silver Samba,['59n7uqUFv9tprlcVU9CoXL'],['Jive Ass Sleepers'],0,False,Solo,1,1,79,...,0.692,0.487,0.070900,0.0860,-13.882,0.0324,0.7140,109.978,0,2015-08-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11098,0xAqhyWvTkfcdFgm7dozMO,(I Wanna) Testify (Mono Single Version),['1Ve5PoI7i5jIIP1xudo6AB'],['The Parliaments'],7,False,Solo,10,1,79,...,0.668,0.702,0.000000,0.0521,-5.750,0.0330,0.7540,105.227,1,2019-08-23
10903,6zhg8TnVKi90ITDzFKwPr0,Swingtown - Alternate Version,['6QtGlUje9TIkLrgPZrESuk'],['Steve Miller Band'],26,False,Solo,5,1,79,...,0.459,0.764,0.000544,0.2440,-6.662,0.0354,0.4390,124.557,1,2019-08-23
5856,2GvkFSjRXV0Ii3uYnHiQ74,Take My Advice,['40PLSl99xXqAQsCQQPmfyM'],['Kym Sims'],0,False,Solo,10,1,73,...,0.737,0.814,0.392000,0.0844,-10.396,0.0307,0.8030,120.010,1,2019-08-23
5080,0lej8CemYKkWVHg77Xe1ky,A Man Holdin' on (To a Woman Lettin’ Go),"['1Fj0R2t4HaJa3oUe8azB8R', '5LDnSV99gbDSsMX3Dc...","['Ty Herndon', 'Anita Cochran']",14,False,Collaboration,7,2,79,...,0.611,0.429,0.000014,0.1120,-8.619,0.0298,0.2370,128.007,1,2019-08-23
